In [4]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [5]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout, Flatten, Activation, BatchNormalization
from keras.layers import RandomContrast, RandomZoom, RandomFlip, RandomRotation, RandomTranslation, RandomCrop, RandomBrightness
from keras.optimizers import Adam, RMSprop, SGD
from keras.models import Sequential
from keras.losses import SparseCategoricalCrossentropy
from keras.callbacks import ModelCheckpoint

AUTOTUNE = tf.data.AUTOTUNE

#### Loading Dataset


In [6]:
def loadImg(filenames, labels, size, resize, rescale):
    images = []
    for i, name in enumerate(filenames):
        try:
            img = tf.keras.preprocessing.image.load_img(name)
        except:
            labels = np.delete(labels, i)
            continue
        img = tf.cast(img, tf.float32)

        if resize:
            img = tf.image.resize(img, (size, size))

        if rescale == 1:
            img = img / 255
        elif rescale == 2:
            img = (img / 127.5) - 1
        else:
            img = tf.cast(img, tf.uint8)

        images.append(img)
    return images, labels


def parseData(filename, delimiter=',', size=False, resize=False, rescale=True):
    if resize:
        if not size:
            raise Exception('Size must be specified when resize is true.')

    images = []
    train_df = pd.read_csv(str(filename)+"_classes.csv", delimiter=delimiter)
    img_dir = np.array(train_df.pop('filename'))
    img_dir = np.ndarray.flatten(img_dir)

    for row, img_path in enumerate(img_dir):
        img_dir[row] = os.path.join(filename, img_path)

    labels = np.array(train_df.idxmax(axis=1).str.strip().astype(
        'category').cat.codes).reshape(-1, 1)

    images, labels = loadImg(img_dir, labels, size, resize, rescale)

    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    return dataset

In [7]:
disease_dict = {0: 'bacterial_blight', 1: 'Blast',
                2: 'Brownspot', 3: 'leaf_scald', 4: 'leaf_burn', 5: 'tungro'}
img_size = 299

In [5]:
with tf.device('/cpu:0'):
    train_dataset = parseData("./dataset/train/", size=img_size,
                              resize=True, rescale=2)
    valid_dataset = parseData("./dataset/valid/", size=img_size,
                              resize=True, rescale=2)

In [17]:
train = train_dataset.shuffle(512).cache(
    'train_cache').batch(32).prefetch(AUTOTUNE)
valid = valid_dataset.cache().batch(32).prefetch(AUTOTUNE)

In [7]:
base_model = tf.keras.applications.inception_v3.InceptionV3(
    include_top=False, input_shape=(img_size, img_size, 3), pooling='max')
base_model.trainable = False

In [ ]:
model = Sequential([
    Input((img_size, img_size, 3)),
    RandomFlip(),
    RandomRotation(factor=0.2),
    # RandomZoom(height_factor=(-0.2, 0.2)),
    base_model,
    Dense(512, 'selu'),
    # Dropout(0.2),
    # Dense(256, 'selu'),
    # Dropout(0.2),
    # Dense(32, 'selu'),
    Dense(len(disease_dict.keys()), 'softmax')
])

model.compile(
    loss=SparseCategoricalCrossentropy(),
    optimizer=Adam(0.0001),
    metrics=['accuracy']
)

bestCB = ModelCheckpoint(filepath='./checkpoint/best/', monitor='val_accuracy',
                         mode='max', verbose=1, save_best_only=True)

model.summary()

In [ ]:
# with tf.device('/cpu:0'):
with tf.device('/gpu:0'):
    history = model.fit(
        train,
        validation_data=valid,
        epochs=10,
        callbacks=bestCB
    )

In [ ]:
model.save("./checkpoint/latest/")

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = history.epoch

plt.plot(epochs, acc, label='Train Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Model's Accuracy")
plt.legend()
plt.show()

plt.plot(epochs, loss, label='Train Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Model's Loss")
plt.legend()
plt.show()

#### Evaluasi Model


In [24]:
# Load iterasi terakhir
# model = tf.keras.models.load_model("./checkpoint/latest/")

# Load terbaik
model = tf.keras.models.load_model("./checkpoint/best/")

In [8]:
with tf.device('/cpu:0'):
    test_dataset = parseData("./dataset/test/", size=img_size,
                             resize=True, rescale=False)

In [ ]:
test = test_dataset.shuffle(128).cache().prefetch(AUTOTUNE)

for image, label in test.take(10):
    image = tf.cast(image, tf.uint8)
    plt.imshow(image)
    plt.show()

    image = (tf.cast(image, tf.float32) / 127.5) - 1
    image = tf.expand_dims(image, axis=0)
    yhat = model.predict(image)
    yhat = np.argmax(yhat, axis=1)
    result = "benar" if yhat == label else "salah"
    print(f"Label asli: {label}\nPrediksi: {yhat} {result}.")